# 🔹 LLM Train Loop: Adım Adım Kod Akışı

LLM train loop’u adım adım inşa edeceğiz ve her adımda iyileştirmeler ekleyeceğiz. Teorideki 5 temel adımı kod seviyesinde ele alacağız.

---

## 1️⃣ Veri ve Batch Hazırlığı
- Dataset ve DataLoader oluşturma
- Veriyi GPU/CPU cihazına taşıma
- Padding ve attention mask kontrolleri

---

## 2️⃣ Forward Pass
- Modeli çağırma
- Teacher forcing uygulanması (decoder kullanılıyorsa)
- Çıkış (`logits`) boyut kontrolü

---

## 3️⃣ Loss Hesaplama
- CrossEntropyLoss (veya label smoothing)
- Padding tokenlarını ignore etme
- Token boyutlarını reshape etme (`[B*T, V]` ve `[B*T]`)

---

## 4️⃣ Backward Pass & Optimize
- `loss.backward()`
- Gradient clipping
- `optimizer.step()`

---

## 5️⃣ Epoch Döngüsü ve Logging
- Toplam loss biriktirme
- Ortalama loss hesaplama
- Opsiyonel: basit progress bar veya logging

---

## 6️⃣ İyileştirmeler (Opsiyonel)
- Mixed precision (AMP) kullanımı
- Gradient accumulation
- Learning rate scheduler

---

> 💡 Not: Önce temel döngüyü çalışır hâle getireceğiz, ardından isteğe bağlı iyileştirmeleri ekleyeceğiz.


----

# 1️⃣: Veri ve Batch Hazırlığı

In [2]:
import torch
from torch.utils.data import DataLoader, Dataset

# -----------------------------
# Örnek Dataset
# -----------------------------
class MyDataset(Dataset):
    def __init__(self, enc_inputs, dec_targets):
        self.enc_inputs = enc_inputs
        self.dec_targets = dec_targets

    def __len__(self):
        return len(self.enc_inputs)

    def __getitem__(self, idx):
        return {
            'input_ids': torch.tensor(self.enc_inputs[idx], dtype=torch.long),
            'target_ids': torch.tensor(self.dec_targets[idx], dtype=torch.long)
        }

# -----------------------------
# Örnek veri
# -----------------------------
enc_inputs = [[1,2,3,4,0,0],[5,6,7,0,0,0]]   # 0 = PAD token
dec_targets = [[1,2,3,4,5,0],[1,2,3,4,0,0]]

# Dataset & DataLoader
dataset = MyDataset(enc_inputs, dec_targets)
batch_size = 2
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# -----------------------------
# GPU/CPU cihazı
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# -----------------------------
# Örnek batch kullanımı
# -----------------------------
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)       # [B, seq_len]
    target_ids = batch['target_ids'].to(device)     # [B, seq_len]
    
    print("Input IDs:", input_ids)
    print("Target IDs:", target_ids)
    break


Input IDs: tensor([[5, 6, 7, 0, 0, 0],
        [1, 2, 3, 4, 0, 0]], device='cuda:0')
Target IDs: tensor([[1, 2, 3, 4, 0, 0],
        [1, 2, 3, 4, 5, 0]], device='cuda:0')


### 🔹 Açıklamalar

* Dataset ve DataLoader:

MyDataset sınıfı, encoder girişleri (input_ids) ve decoder hedeflerini (target_ids) alıyor.

DataLoader mini-batch oluşturur ve shuffle ile rastgele sırayla verir.

* Cihaz (device) seçimi:

torch.device ile GPU varsa oraya, yoksa CPU’ya taşır.

* Batch çekme:

Her iterasyonda input_ids ve target_ids batch olarak alınır.

Modelin forward pass’ine hazır hâle gelir.

* Padding kontrolü:

Örnek veride 0 PAD token olarak kullanıldı.

Daha sonra loss hesaplamada ignore edilecek.

---
# 2️⃣: Forward Pass

In [3]:
import torch
import torch.nn as nn

# -----------------------------
# Örnek Model (Encoder-Decoder)
# -----------------------------
class SimpleSeq2Seq(nn.Module):
    def __init__(self, vocab_size=10, embed_dim=16, hidden_dim=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, enc_input, dec_input):
        # Encoder
        enc_emb = self.embedding(enc_input)
        _, (h, c) = self.encoder(enc_emb)
        
        # Decoder (Teacher Forcing)
        dec_emb = self.embedding(dec_input)
        dec_out, _ = self.decoder(dec_emb, (h, c))
        
        # Logits
        logits = self.fc(dec_out)
        return logits

# -----------------------------
# Model ve cihaz
# -----------------------------
vocab_size = 10
model = SimpleSeq2Seq(vocab_size=vocab_size).to(device)

# -----------------------------
# Örnek Forward Pass
# -----------------------------
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    target_ids = batch['target_ids'].to(device)
    
    # Teacher forcing için decoder input: target_ids kaydırılmış
    dec_input = target_ids[:, :-1]
    
    # Forward
    logits = model(input_ids, dec_input)  # [B, seq_len-1, vocab_size]
    
    print("Logits shape:", logits.shape)
    break


Logits shape: torch.Size([2, 5, 10])


### 🔹 Açıklamalar

* Embedding & Encoder:

input_ids önce embedding katmanından geçer.

Encoder LSTM, hidden ve cell state üretir.

* Decoder & Teacher Forcing:

dec_input = target_ids[:, :-1] → hedef diziyi bir adım kaydırarak veriyoruz.

Decoder, hidden state’i encoder’dan alır.

Bu şekilde model, bir adım ileriyi tahmin etmeyi öğrenir.

* Logits:

Decoder çıkışı Linear ile vocab boyutuna dönüştürülür.

Shape: [batch_size, seq_len-1, vocab_size]

Loss hesaplamaya hazır.

---
# 3️⃣: Loss Hesaplama

In [4]:
import torch.nn as nn

# -----------------------------
# Loss fonksiyonu
# -----------------------------
PAD_TOKEN = 0  # padding token
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN)

# -----------------------------
# Örnek batch ve logits
# -----------------------------
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    target_ids = batch['target_ids'].to(device)
    
    dec_input = target_ids[:, :-1]  # decoder input (teacher forcing)
    dec_target = target_ids[:, 1:]  # gerçek hedef
    
    # Forward Pass
    logits = model(input_ids, dec_input)  # [B, seq_len-1, vocab_size]
    
    # Reshape logits ve target: [B*T, V] ve [B*T]
    B, T, V = logits.shape
    logits_flat = logits.reshape(B*T, V)
    target_flat = dec_target.reshape(B*T)
    
    # Loss
    loss = criterion(logits_flat, target_flat)
    
    print("Loss:", loss.item())
    break


Loss: 2.2987773418426514


### 🔹 Açıklamalar

* Decoder Target:

dec_target = target_ids[:, 1:] → Teacher forcing için bir adım kaydırılmış hedef.

- Shape Dönüşümü:

CrossEntropyLoss [N, C] ve [N] boyutunda bekler.

[B, T, V] → [B*T, V] ve [B, T] → [B*T]

- Padding Tokenları:

ignore_index=PAD_TOKEN sayesinde paddingler loss hesabına dahil edilmez.

Böylece model yalnızca gerçek tokenlar üzerinden öğrenir.

- Loss Hazır:

Bu loss artık backward pass için kullanılabilir.

---

# 4️⃣: Backward Pass & Optimize

In [5]:
# -----------------------------
# Optimizer
# -----------------------------
import torch.optim as optim

optimizer = optim.AdamW(model.parameters(), lr=5e-4)

# -----------------------------
# Örnek Backward Pass
# -----------------------------
for batch in train_loader:
    input_ids = batch['input_ids'].to(device)
    target_ids = batch['target_ids'].to(device)
    
    dec_input = target_ids[:, :-1]
    dec_target = target_ids[:, 1:]
    
    # Forward
    logits = model(input_ids, dec_input)
    B, T, V = logits.shape
    logits_flat = logits.reshape(B*T, V)
    target_flat = dec_target.reshape(B*T)
    
    # Loss
    loss = criterion(logits_flat, target_flat)
    
    # -----------------------------
    # Backward & Optimize
    # -----------------------------
    optimizer.zero_grad()            # gradyanları sıfırla
    loss.backward()                  # backward pass
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # gradient clipping
    optimizer.step()                 # ağırlıkları güncelle
    
    print("Updated weights for this batch, Loss:", loss.item())
    break


Updated weights for this batch, Loss: 2.2987773418426514


### 🔹 Açıklamalar

* Gradyan Sıfırlama

optimizer.zero_grad() → Önceki batch’in gradyanları temizlenir.

* Backward Pass

loss.backward() → Model ağı boyunca gradyanlar hesaplanır.

* Gradient Clipping

clip_grad_norm_ ile gradyan patlaması önlenir.

Özellikle LLM ve derin ağlarda kritik.

* Optimizer Step

optimizer.step() → Hesaplanan gradyanlar ile ağırlıklar güncellenir.

----
# 5️⃣: Epoch Döngüsü ve Logging

In [6]:
# -----------------------------
# Hyperparameters
# -----------------------------
epochs = 3
batch_size = 2

# -----------------------------
# Train Loop
# -----------------------------
for epoch in range(epochs):
    model.train()                     # train moduna al
    total_loss = 0.0
    
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        
        dec_input = target_ids[:, :-1]
        dec_target = target_ids[:, 1:]
        
        # Forward
        logits = model(input_ids, dec_input)
        B, T, V = logits.shape
        logits_flat = logits.reshape(B*T, V)
        target_flat = dec_target.reshape(B*T)
        
        # Loss
        loss = criterion(logits_flat, target_flat)
        
        # Backward & Optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] Average Loss: {avg_loss:.4f}")


Epoch [1/3] Average Loss: 2.2899
Epoch [2/3] Average Loss: 2.2810
Epoch [3/3] Average Loss: 2.2722


### 🔹 Açıklamalar

* Epoch Döngüsü

for epoch in range(epochs) → Model veriyi kaç kez görecek.

model.train() → dropout, batchnorm gibi katmanları training moduna alır.

* Batch Döngüsü

train_loader ile mini-batch’ler işlenir.

Forward, loss, backward ve optimize adımları batch başına uygulanır.

* Loss Biriktirme & Ortalama

total_loss += loss.item() → Batch’ten batch’e toplam loss birikir.

avg_loss = total_loss / len(train_loader) → epoch sonunda ortalama loss yazdırılır.

* Logging

Basit print ile loss gözlemlenebilir.

İleri seviye: tqdm veya tensorboard ile görselleştirilebilir.

---

## KODUN DÜZENLENMİŞ VE UYARLANMIŞ HALİ ;
```python
# -----------------------------
# Hyperparameters
# -----------------------------
epochs = 3
batch_size = 2
max_grad_norm = 1.0

# -----------------------------
# Train Loop (Düzenli)
# -----------------------------
for epoch in range(epochs):
    model.train()                     # Training moduna al
    total_loss = 0.0
    
    for batch_idx, batch in enumerate(train_loader):
        # -------------------------
        # 1️⃣ Veri ve batch hazırlığı
        # -------------------------
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        dec_input = target_ids[:, :-1]   # teacher forcing için
        dec_target = target_ids[:, 1:]   # gerçek hedef
        
        # -------------------------
        # 2️⃣ Forward Pass
        # -------------------------
        logits = model(input_ids, dec_input)  # [B, seq_len-1, vocab_size]
        
        # -------------------------
        # 3️⃣ Loss Hesaplama
        # -------------------------
        B, T, V = logits.shape
        logits_flat = logits.reshape(B*T, V)
        target_flat = dec_target.reshape(B*T)
        loss = criterion(logits_flat, target_flat)
        
        # -------------------------
        # 4️⃣ Backward Pass & Optimize
        # -------------------------
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        
        # -------------------------
        # 5️⃣ Logging
        # -------------------------
        total_loss += loss.item()
        if (batch_idx + 1) % 1 == 0:  # Batch bazlı print
            print(f"Epoch [{epoch+1}/{epochs}] | Batch [{batch_idx+1}/{len(train_loader)}] | Loss: {loss.item():.4f}")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}\n")


---

## Teacher Forcing'i hiç duydunuz mu ? Nedir Teacher Forcing ? 

----
# 🔹 Teacher Forcing Nedir?

### Tanım:
* Decoder’ı eğitirken, modelin kendi önceki tahminlerini kullanmak yerine gerçek hedef tokenları bir sonraki adım için girdi olarak vermek yöntemidir.

### Amaç:

* Uzun dizilerde hata birikimini önlemek

* Öğrenmeyi hızlandırmak

* Modelin doğru diziyi daha hızlı öğrenmesini sağlamak

## 🔹 Nasıl Çalışır?

**Normal seq2seq tahmininde:**

> Tahmin_0 -> Tahmin_1 -> Tahmin_2 -> ...


* Her adımda decoder, önceki tahminini kullanır.

* Eğer model yanlış bir tahmin yaparsa, hata birikir ve sonraki adımlar da yanlış olur.

**Teacher forcing kullanıldığında:**

 > Gerçek_0 -> Gerçek_1 -> Gerçek_2 -> ...


* Decoder’a bir adım kaydırılmış gerçek hedef token verilir.

* Böylece model her adımda doğru bağlamı görür.

## 🔹 Kod Örneği
```python
dec_input = target_ids[:, :-1]  # Teacher forcing: bir adım kaydırılmış hedef
dec_target = target_ids[:, 1:]  # Loss hesaplamak için gerçek hedef

logits = model(input_ids, dec_input)
```


* dec_input → Decoder’a verilecek giriş (teacher forcing uygulanmış)

* dec_target → Loss hesaplamak için gerçek hedef tokenlar

* Model, dec_input üzerinden tahmin yapar ve dec_target ile karşılaştırır.

## 🔹 Avantajları ve Dezavantajları
### Avantajlar:

* Hata birikimi azalır

* Eğitim daha hızlı ve stabil

* Özellikle uzun dizilerde çok işe yarar

### Dezavantajlar:

* Train-test farkı yaratabilir:

* Test sırasında model kendi tahminlerini kullanır.

* Bu nedenle eğitimde gördüğü bağlam ile testteki bağlam farklı olabilir.

#### **Çözüm: Scheduled sampling veya kısmi teacher forcing**

In [ ]:
# -----------------------------
# Hyperparameters
# -----------------------------
epochs = 3
batch_size = 2
max_grad_norm = 1.0
teacher_forcing = True  # True = teacher forcing, False = kendi tahminini kullan

# -----------------------------
# Train Loop
# -----------------------------
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    
    for batch_idx, batch in enumerate(train_loader):
        # 1️⃣ Veri & batch hazırlığı
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        
        dec_input = target_ids[:, :-1]    # teacher forcing için
        dec_target = target_ids[:, 1:]    # gerçek hedef
        
        # 2️⃣ Forward Pass (teacher forcing opsiyonel)
        logits = model(input_ids, dec_input=dec_input, target_ids=target_ids,
                       teacher_forcing=teacher_forcing)
        
        # 3️⃣ Loss Hesaplama
        B, T, V = logits.shape
        logits_flat = logits.reshape(B*T, V)
        target_flat = dec_target.reshape(B*T)
        loss = criterion(logits_flat, target_flat)
        
        # 4️⃣ Backward & Optimize
        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        
        # 5️⃣ Logging
        total_loss += loss.item()
        if (batch_idx + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] | Batch [{batch_idx+1}/{len(train_loader)}] | Loss: {loss.item():.4f}")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}\n")

----
# Şu ana kadar bir train işleminin aşamalarını ve temelden işleyişini gördük.Sıra bu işleyişi daha optimize ve daha maliyetsiz hale getirmek.
---

## 🔹 AMP (Automatic Mixed Precision) Nedir?

Amaç:

* Modelin bazı hesaplamalarını float16 (half precision) ile yapıp, diğerlerini float32 olarak tutmak

* GPU belleğini ve hesaplama süresini optimize etmek

Nasıl çalışır:

* Forward ve backward pass’te torch.cuda.amp.autocast() kullanılır

* Gradient update sırasında torch.cuda.amp.GradScaler ile gradyanlar ölçeklenir

## 🔹 GPU Optimizasyonu

* Cihaz seçimi: device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

* Tüm tensor ve model GPU’ya taşınır: .to(device)

* Batch size ve precision GPU’ya uygun ayarlanır

# 🔹 AMP ile Düzenlenmiş Train Loop

In [ ]:
from torch.cuda.amp import autocast, GradScaler

scaler = GradScaler()  # Gradient scaler

epochs = 3
max_grad_norm = 1.0
teacher_forcing = True

for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        dec_input = target_ids[:, :-1]
        dec_target = target_ids[:, 1:]
        
        optimizer.zero_grad()
        
        # -----------------------------
        # Forward + Loss (AMP ile)
        # -----------------------------
        with autocast():  # mixed precision context
            logits = model(input_ids, dec_input=dec_input,
                           target_ids=target_ids,
                           teacher_forcing=teacher_forcing)
            B, T, V = logits.shape
            logits_flat = logits.reshape(B*T, V)
            target_flat = dec_target.reshape(B*T)
            loss = criterion(logits_flat, target_flat)
        
        # -----------------------------
        # Backward + Optimize (AMP ile)
        # -----------------------------
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
        if (batch_idx + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] | Batch [{batch_idx+1}/{len(train_loader)}] | Loss: {loss.item():.4f}")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}\n")


### 🔹 Açıklamalar

#### **autocast()**

* Forward ve loss hesaplamasını float16/32 karma olarak yapar

* Hız ve VRAM tasarrufu sağlar

#### **GradScaler**

* Küçük gradyanları ölçekleyerek underflow’u önler

- scaler.scale(loss).backward() ve scaler.step(optimizer) ile birlikte çalışır

#### **GPU kullanımı**

* Model ve tüm tensorlar .to(device) ile GPU’ya taşınır

* Büyük batch’lerde eğitim verimli olur

---
# Bir sonraki adım olarak gradient accumulation ve scheduler ekleyip, büyük batch’leri küçük GPU belleği ile çalıştırabilecek şekilde loop’u geliştireceğiz.
---

## 🔹 Gradient Accumulation Nedir?

#### **Amaç:**

* Küçük GPU belleği olan makinelerde, büyük batch’i parçalara bölerek işlem yapmak

* Her mini-batch için backward yapıp, birkaç mini-batch sonra optimizer step atmak

#### **Nasıl çalışır:**

* accumulation_steps = N → N mini-batch biriktirilir

* loss.backward() her mini-batch için yapılır

* optimizer.step() ve scaler.update() sadece N adımda bir yapılır

## 🔹 Scheduler Nedir?

* Öğrenme oranını dinamik olarak ayarlayan mekanizma

* Örnek: StepLR, CosineAnnealingLR, OneCycleLR

* Özellikle LLM’lerde stabil eğitim için önemlidir

# 🔹 Optimized Train Loop (AMP + Gradient Accumulation + Scheduler)

In [ ]:
from torch.optim.lr_scheduler import StepLR

# -----------------------------
# Parametreler
# -----------------------------
epochs = 3
max_grad_norm = 1.0
teacher_forcing = True
accumulation_steps = 2  # 2 mini-batch biriktir
scaler = GradScaler()

# Scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)  # örnek

# -----------------------------
# Train Loop
# -----------------------------
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    
    optimizer.zero_grad()
    
    for batch_idx, batch in enumerate(train_loader):
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        dec_input = target_ids[:, :-1]
        dec_target = target_ids[:, 1:]
        
        with autocast():
            logits = model(input_ids, dec_input=dec_input,
                           target_ids=target_ids,
                           teacher_forcing=teacher_forcing)
            B, T, V = logits.shape
            logits_flat = logits.reshape(B*T, V)
            target_flat = dec_target.reshape(B*T)
            loss = criterion(logits_flat, target_flat) / accumulation_steps  # loss scaling
        
        scaler.scale(loss).backward()
        total_loss += loss.item() * accumulation_steps  # orijinal loss
        
        # Gradient accumulation step
        if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()  # LR update
        
        # Logging
        if (batch_idx + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] | Batch [{batch_idx+1}/{len(train_loader)}] | Loss: {loss.item()*accumulation_steps:.4f}")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}\n")


## 🔹 Açıklamalar

### **Loss scaling**

* loss / accumulation_steps → gradyanlar birikince toplam loss doğru olur

### **Accumulation**

* if (batch_idx + 1) % accumulation_steps == 0 → optimizer ve scaler update

### **Scheduler**

* scheduler.step() → learning rate her accumulation step veya batch sonunda güncellenebilir

* AMP + GPU

* autocast() ve GradScaler() ile VRAM ve hız optimizasyonu devam eder

---
# O zaman şimdi Top-K ve Top-P sampling destekli inference kısmını ekleyelim. Bu, modelin eğitildikten sonra çeşitli ve mantıklı çıktılar üretmesini sağlar.
---

In [12]:
import torch
import torch.nn as nn

# -----------------------------
# Minik Seq2Seq Model (Test için)
# -----------------------------
class MiniSeq2Seq(nn.Module):
    def __init__(self, vocab_size=20, embed_dim=16, hidden_dim=32):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.encoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.decoder = nn.LSTM(embed_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, enc_input, dec_input=None, target_ids=None, teacher_forcing=True, max_len=20):
        enc_emb = self.embedding(enc_input)
        _, (h, c) = self.encoder(enc_emb)
        
        # -----------------------------
        # Teacher forcing açık
        # -----------------------------
        if teacher_forcing and dec_input is not None:
            dec_emb = self.embedding(dec_input)
            dec_out, _ = self.decoder(dec_emb, (h, c))
            logits = self.fc(dec_out)
        
        # -----------------------------
        # Teacher forcing kapalı (inference)
        # -----------------------------
        else:
            if target_ids is not None:
                start_token = target_ids[:, 0].unsqueeze(1)
            else:
                start_token = enc_input[:, -1].unsqueeze(1)  # son token veya <BOS>
            
            inputs = start_token
            outputs = []
            hidden = (h, c)
            
            for _ in range(max_len):
                emb = self.embedding(inputs)
                out, hidden = self.decoder(emb, hidden)
                logit = self.fc(out)
                outputs.append(logit)
                inputs = logit.argmax(-1)  # kendi tahminini input olarak kullan
            
            logits = torch.cat(outputs, dim=1)
        
        return logits


# -----------------------------
# Model, Device ve Test
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MiniSeq2Seq(vocab_size=20).to(device)


## 🔹 Sampling (Inference) Fonksiyonu

In [13]:
@torch.no_grad()
def generate(model, input_ids, max_len=20, top_k=50, top_p=0.9, temperature=1.0, device='cuda'):
    """
    Top-K / Top-P sampling ile token üretme
    """
    model.eval()
    input_ids = input_ids.to(device)
    
    generated = input_ids  # başlangıç token
    B = input_ids.size(0)
    
    for _ in range(max_len):
        # decoder kendi tahminini input olarak kullanacak, teacher forcing kapalı
        logits = model(generated, dec_input=None, target_ids=None, teacher_forcing=False, max_len=1)
        next_token_logits = logits[:, -1, :] / temperature
        
        # -----------------------------
        # Top-K Sampling
        # -----------------------------
        if top_k > 0:
            top_k_vals, top_k_idx = torch.topk(next_token_logits, top_k)
            probs = torch.zeros_like(next_token_logits).scatter_(-1, top_k_idx, F.softmax(top_k_vals, dim=-1))
        else:
            probs = F.softmax(next_token_logits, dim=-1)
        
        # -----------------------------
        # Top-P Sampling
        # -----------------------------
        if top_p < 1.0:
            sorted_probs, sorted_idx = torch.sort(probs, descending=True)
            cumulative_probs = torch.cumsum(sorted_probs, dim=-1)
            sorted_idx_to_remove = cumulative_probs > top_p
            sorted_probs[sorted_idx_to_remove] = 0
            sorted_probs /= sorted_probs.sum(dim=-1, keepdim=True)
            probs = torch.zeros_like(probs).scatter_(-1, sorted_idx, sorted_probs)
        
        # -----------------------------
        # Token seçimi
        # -----------------------------
        next_token = torch.multinomial(probs, num_samples=1)
        generated = torch.cat([generated, next_token], dim=1)
    
    return generated


## 🔹 Açıklamalar

### > teacher_forcing=False

* Inference sırasında model kendi tahminini bir sonraki input olarak kullanır

### > Temperature

* temperature < 1 → dağılım keskinleşir → deterministik

* temperature > 1 → dağılım düzleşir → daha çeşitli çıktılar

### > Top-K Sampling

* En yüksek K olasılıklı tokenlar arasından rastgele seçim

### > Top-P (Nucleus) Sampling

* Kümülatif olasılığı P olan token setinden rastgele seçim

* Dinamik ve mantıklı token seçimi sağlar

### > Multinomial

* Token olasılıklarına göre rastgele seçim yapılır

In [14]:
start_token = torch.tensor([[1]])  # batch size=1, start token

generated_topk = generate(model, start_token, max_len=10, top_k=5, top_p=1.0, temperature=1.0, device=device)
print("Generated sequence (Top-K):", generated_topk.tolist())

generated_topp = generate(model, start_token, max_len=10, top_k=0, top_p=0.9, temperature=1.0, device=device)
print("Generated sequence (Top-P):", generated_topp.tolist())


Generated sequence (Top-K): [[1, 2, 3, 10, 2, 18, 3, 10, 3, 10, 12]]
Generated sequence (Top-P): [[1, 4, 5, 4, 12, 18, 7, 7, 3, 18, 6]]


# 🔹 Minik Tokenizer Örneği

In [15]:
# -----------------------------
# Mini tokenizer
# -----------------------------
id2token = {i: f"tok{i}" for i in range(20)}  # token ID → string
token2id = {v:k for k,v in id2token.items()}

def decode(token_ids):
    """
    Token ID listesini string hâline çevirir
    """
    if isinstance(token_ids, torch.Tensor):
        token_ids = token_ids.tolist()
    # batch support: batch_size x seq_len
    if isinstance(token_ids[0], list):
        return [" ".join(id2token[tok] for tok in seq) for seq in token_ids]
    else:
        return " ".join(id2token[tok] for tok in token_ids)


## 🔹 Test: Top-K ve Top-P Sampling

In [16]:
# Örnek start token
start_token = torch.tensor([[1]])

# Top-K sampling
generated_topk = generate(model, start_token, max_len=10, top_k=5, top_p=1.0, temperature=1.0, device=device)
print("Generated sequence (Top-K IDs):", generated_topk.tolist())
print("Generated sequence (Top-K Decoded):", decode(generated_topk))

# Top-P sampling
generated_topp = generate(model, start_token, max_len=10, top_k=0, top_p=0.9, temperature=1.0, device=device)
print("Generated sequence (Top-P IDs):", generated_topp.tolist())
print("Generated sequence (Top-P Decoded):", decode(generated_topp))


Generated sequence (Top-K IDs): [[1, 3, 18, 2, 3, 7, 16, 18, 10, 18, 3]]
Generated sequence (Top-K Decoded): ['tok1 tok3 tok18 tok2 tok3 tok7 tok16 tok18 tok10 tok18 tok3']
Generated sequence (Top-P IDs): [[1, 8, 17, 18, 12, 12, 10, 8, 12, 3, 6]]
Generated sequence (Top-P Decoded): ['tok1 tok8 tok17 tok18 tok12 tok12 tok10 tok8 tok12 tok3 tok6']


----

## Yukarıda anlatılan işlemler doğrultusunda oluşturulan "TRAİN_LOOP" aşağıdaki gibidir ;

In [ ]:
import torch
import torch.nn as nn
from torch.cuda.amp import autocast, GradScaler
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR

# -----------------------------
# Parametreler
# -----------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs = 3
max_grad_norm = 1.0
teacher_forcing = True
accumulation_steps = 2  # büyük batch simülasyonu
learning_rate = 1e-3

# -----------------------------
# Model, Optimizer, Scaler, Scheduler
# -----------------------------
model = MiniSeq2Seq(vocab_size=20).to(device)
optimizer = Adam(model.parameters(), lr=learning_rate)
scaler = GradScaler()
scheduler = StepLR(optimizer, step_size=1, gamma=0.9)  # örnek scheduler
criterion = nn.CrossEntropyLoss()

# -----------------------------
# Train Loop
# -----------------------------
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    optimizer.zero_grad()
    
    for batch_idx, batch in enumerate(train_loader):  # train_loader = DataLoader objesi
        input_ids = batch['input_ids'].to(device)
        target_ids = batch['target_ids'].to(device)
        dec_input = target_ids[:, :-1]
        dec_target = target_ids[:, 1:]
        
        # -----------------------------
        # Forward + Loss (AMP)
        # -----------------------------
        with autocast():
            logits = model(input_ids, dec_input=dec_input,
                           target_ids=target_ids,
                           teacher_forcing=teacher_forcing)
            
            B, T, V = logits.shape
            logits_flat = logits.reshape(B*T, V)
            target_flat = dec_target.reshape(B*T)
            loss = criterion(logits_flat, target_flat) / accumulation_steps  # scaled for accumulation
        
        # -----------------------------
        # Backward (AMP + Grad Accumulation)
        # -----------------------------
        scaler.scale(loss).backward()
        total_loss += loss.item() * accumulation_steps
        
        if (batch_idx + 1) % accumulation_steps == 0 or (batch_idx + 1) == len(train_loader):
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()
        
        # Logging
        if (batch_idx + 1) % 1 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] | Batch [{batch_idx+1}/{len(train_loader)}] | Loss: {loss.item()*accumulation_steps:.4f}")
    
    avg_loss = total_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{epochs}] completed. Average Loss: {avg_loss:.4f}\n")


In [19]:
# -----------------------------
# Örnek start token
# -----------------------------
start_token = torch.tensor([[1]]).to(device)

# -----------------------------
# Top-K Sampling
# -----------------------------
generated_topk = generate(model, start_token, max_len=10, top_k=5, top_p=1.0, temperature=1.0, device=device)
print("Generated sequence (Top-K IDs):", generated_topk.tolist())
print("Generated sequence (Top-K Decoded):", decode(generated_topk))

# -----------------------------
# Top-P (Nucleus) Sampling
# -----------------------------
generated_topp = generate(model, start_token, max_len=10, top_k=0, top_p=0.9, temperature=1.0, device=device)
print("Generated sequence (Top-P IDs):", generated_topp.tolist())
print("Generated sequence (Top-P Decoded):", decode(generated_topp))


Generated sequence (Top-K IDs): [[1, 2, 4, 7, 7, 0, 19, 7, 12, 16, 3]]
Generated sequence (Top-K Decoded): ['tok1 tok2 tok4 tok7 tok7 tok0 tok19 tok7 tok12 tok16 tok3']
Generated sequence (Top-P IDs): [[1, 19, 5, 2, 11, 9, 18, 10, 8, 12, 12]]
Generated sequence (Top-P Decoded): ['tok1 tok19 tok5 tok2 tok11 tok9 tok18 tok10 tok8 tok12 tok12']
